<h2>1.   Analisando Dados de E-Commerce</h2><br>
Tudo aqui foi escrito por Vìtor Albuquerque (:<br>

Este é um conjunto de dados público de comércio eletrônico brasileiro. <br>
Contém informações de 100 mil pedidos de 2016 a 2018 feitos em vários marketplaces no Brasil. Seus recursos permitem visualizar um pedido de várias dimensões.

Neste notebook, utilizarei a linguagem SQL + Python para trazer alguns insights importantes para o negócio, que aliados à conceitos básicos de métricas comerciais, responderão a questões como:<br>
* Performance de Departamentos<br>
* Performance de Meios de Pagamento<br>
* Diarização de Vendas<br>



Primeiramente, importamos as bibliotecas que vamos utilizar neste notebook.

In [12]:
import pandas as pd
import numpy as np
import sqlite3 as sql

Logo após, é necessário realizar a conexão ao banco de dados 'db', onde residem as tabelas:<br><br>
olist_customers_dataset<br>
olist_geolocation_dataset<br>
olist_order_items_dataset<br>
olist_order_payments_dataset<br>
olist_order_reviews_dataset<br>
olist_orders_dataset<br>
olist_products_dataset<br>
olist_sellers_dataset<br>
product_category_name_translation<br>

In [13]:
database = 'db.db'
connection = sql.connect(database)

<h2>2. Consultando Informações de Departamentos</h2><br>
Departamentos comerciais geralmente trazem consigo importantes informações.<br>
Neste caso, vamos construir uma query que retornará o TPV (TOTAL TRANSACIONADO), TICKET MÉDIO e QUANTIDADE DE PEDIDOS para cada um dos departamentos da nossa base.<br>
Para tal tarefa, vamos precisar consultar 3 tabelas distintas, utilizando a clásula LEFT JOIN.<br>
Além disso, para fins estéticos, vamos substituir os símbolos underline por espaços, e também padronizaremos os nomes das categorias para que fiquem todos em maiúsculo.

In [30]:
query = '''

select 
    replace(upper(t1.product_category_name),'_', ' ') as Categoria
    , count(distinct t2.order_id) as Qtd_Clientes
    , sum(t3.payment_value) as TPV
    , round(sum(t3.payment_value) / count(distinct t2.order_id),2) as Ticket_Medio
    
    from olist_products_dataset as t1
    left join olist_order_items_dataset as t2
    on t1.product_id = t2.product_id
    left join olist_order_payments_dataset as t3
    on t2.order_id = t3.order_id
group by 1
order by 1

'''
df = pd.read_sql_query(query, connection)
display(df)

,Categoria,Qtd_Clientes,TPV,Ticket_Medio
0,None,1451,252801.71,174.23
1,AGRO INDUSTRIA E COMERCIO,182,118730.61,652.37
2,ALIMENTOS,450,46428.67,103.17
3,ALIMENTOS BEBIDAS,227,27289.73,120.22
4,ARTES,202,30992.93,153.43
...,...,...,...,...
69,SINALIZACAO E SEGURANCA,140,68488.05,489.20
70,TABLETS IMPRESSAO IMAGEM,79,10042.93,127.13
71,TELEFONIA,4199,486882.05,115.95
72,TELEFONIA FIXA,217,207010.26,953.96


Legal! Quer saber qual o top 5 departamentos por TPV?

In [33]:
query = '''

select 
    replace(upper(t1.product_category_name),'_', ' ') as Categoria
    , count(distinct t2.order_id) as Qtd_Clientes
    , sum(t3.payment_value) as TPV
    , round(sum(t3.payment_value) / count(distinct t2.order_id),2) as Ticket_Medio
    
    from olist_products_dataset as t1
    left join olist_order_items_dataset as t2
    on t1.product_id = t2.product_id
    left join olist_order_payments_dataset as t3
    on t2.order_id = t3.order_id
group by 1
order by 3 desc
limit 5

'''
df = pd.read_sql_query(query, connection)
display(df)

,Categoria,Qtd_Clientes,TPV,Ticket_Medio
0,CAMA MESA BANHO,9417,1712553.67,181.86
1,BELEZA SAUDE,8836,1657373.12,187.57
2,INFORMATICA ACESSORIOS,6689,1585330.45,237.01
3,MOVEIS DECORACAO,6449,1430176.39,221.77
4,RELOGIOS PRESENTES,5624,1429216.68,254.13


Agora que sabemos quais os departamentos em que os clientes compram mais, é interessante saber qual o share que esses departamentos representam considerando o total de compras geral.<br>
No comando abaixo, inclurei uma coluna para este cálculo.

In [49]:
query = '''

with tb_a as(
    select 
        replace(upper(t1.product_category_name),'_', ' ') as Categoria
        , count(distinct t2.order_id) as Qtd_Clientes
        , sum(t3.payment_value) as TPV
        , round(sum(t3.payment_value) / count(distinct t2.order_id),2) as Ticket_Medio
        , round((select sum(payment_value) from olist_order_payments_dataset),2) as TPV_TOTAL
        from olist_products_dataset as t1
        left join olist_order_items_dataset as t2
        on t1.product_id = t2.product_id
        left join olist_order_payments_dataset as t3
        on t2.order_id = t3.order_id
    group by 1
    order by 3 desc
    limit 5
),

tb_b as (
    select
        Categoria
        , Qtd_Clientes
        , TPV
        , Ticket_Medio
        , round((TPV / TPV_TOTAL)*100,2)as Share
    from tb_a
)

select * from tb_b

'''

df = pd.read_sql_query(query, connection)
display(df)

,Categoria,Qtd_Clientes,TPV,Ticket_Medio,Share
0,CAMA MESA BANHO,9417,1712553.67,181.86,10.70
1,BELEZA SAUDE,8836,1657373.12,187.57,10.35
2,INFORMATICA ACESSORIOS,6689,1585330.45,237.01,9.90
3,MOVEIS DECORACAO,6449,1430176.39,221.77,8.93
4,RELOGIOS PRESENTES,5624,1429216.68,254.13,8.93


Agora que sabemos que o departamento CAMA MESA BANHO é o que possui maior share (participação) no lucro da empresa, vamos entrar mais a fundo nesta categoria.<br>
Que tal verificarmos qual o estado do Brasil que lidera nas compras?

In [70]:
query = '''

with tb_a as (
    select 
        t2.order_id
        , replace(upper(t1.product_category_name),'_', ' ') as Categoria
        , sum(t3.payment_value) as TPV   
        from olist_products_dataset as t1
        left join olist_order_items_dataset as t2
        on t1.product_id = t2.product_id
        left join olist_order_payments_dataset as t3
        on t2.order_id = t3.order_id

    group by 1,2
    order by 1,2
),

tb_b as (

    select 
        t1.*
        , t2.customer_id
        , t3.customer_state as Estado
        , round((select sum(payment_value) from olist_order_payments_dataset where Categoria = 'CAMA MESA BANHO'),2) as TPV_TOTAL
    
    from tb_a as t1
    left join olist_orders_dataset as t2
    on t1.order_id = t2.order_id
    left join olist_customers_dataset as t3
    on t2.customer_id = t3.customer_id

)

select
    Categoria
    , Estado
    , count(distinct order_id) as Qtd_Clientes
    , sum(TPV) as TPV
    , round(sum(TPV) / count(order_id),2) as Ticket_Medio
    , round((sum(TPV) / TPV_TOTAL)*100, 2) as Share

from tb_b
where Categoria = 'CAMA MESA BANHO'
group by 1,2
order by 3 desc
limit 5

'''
df = pd.read_sql_query(query, connection)
display(df)

,Categoria,Estado,Qtd_Clientes,TPV,Ticket_Medio,Share
0,CAMA MESA BANHO,SP,4416,765517.23,173.35,4.78
1,CAMA MESA BANHO,RJ,1393,247744.47,177.85,1.55
2,CAMA MESA BANHO,MG,1129,218010.61,193.10,1.36
3,CAMA MESA BANHO,RS,540,93708.68,173.53,0.59
4,CAMA MESA BANHO,PR,402,76691.14,190.77,0.48


<h2>3. Consultando Meios de Pagamento</h2><br>
Vamos analisar quais os meios de pagamento utilizados em e-commerce.<br>
Na query abaixo iremos observar O TPV por forma de pagamento.<br>
Também para fins de padronização, vamos formatar as escritas dessas informações.

In [94]:
query = '''

select
     case 
        when payment_type = 'credit_card' then 'Credito' 
        when payment_type = 'debit_card' then 'Debito'
        when payment_type = 'boleto' then 'Boleto' 
        when payment_type = 'voucher' then 'Cupom'
        when payment_type = 'not_defined' then 'Nao Especificado'
        end as Meio_Pagamento
    , round(sum(payment_value),2) as TPV

from olist_order_payments_dataset
group by 1
order by 2 desc

'''
df = pd.read_sql_query(query, connection)
display(df)

,Meio_Pagamento,TPV
0,Credito,12542084.19
1,Boleto,2869361.27
2,Cupom,379436.87
3,Debito,217989.79
4,Nao Especificado,0.00


<h2>4. Consultando Compras por Data</h2><br>
A abertura por período é um dos principais métodos de análise de um negócio.<br>
Na query abaixo iremos observar o número de compras nas agregações de mês e ano.<br>